# Census & MBS: Combine Datasets

1. MBS data was separate into 3 separate datasets, National, National-SAGroup and State Level
2. Census and MBS State datasets were combined using year and SA3 area to create a complete combined dataset
3. Combined census and MBS State dataset contained significant number of blank values as demographic data for Service level 2 and 3 did not exist in the MBS dataset
4. Created 2 additional dataset by subsetting combined dataset into service level 1 and service 2+3 dataset. 

In [1184]:
# import libraries
import pandas as pd
import numpy as np
import os

pd.set_option("display.max_rows", 120)

## Import & Ready MBS Combined Dataset

In [1185]:
# import the transformed mbs file and assign to a dataframe

# setup path to original dataset
path = r"/Users/patel/Documents/CF-Data Anaylst Course/portfolio_projects/mbs_analysis/datasets/"

df_mbs_2014_23 = pd.read_pickle(
    os.path.join(path, "clean_datasets/mbs_data/2014-22_phc_combined_mbs.pkl")
)
df_mbs_2014_23.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258535 entries, 0 to 258534
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   Year                                          258535 non-null  int64  
 1   StateTerritory                                258535 non-null  object 
 2   GeographicCode                                258535 non-null  object 
 3   GeographicAreaName                            258535 non-null  object 
 4   GeographicGroup                               258535 non-null  object 
 5   ServiceLevel                                  258535 non-null  object 
 6   Service                                       258535 non-null  object 
 7   DemographicGroup                              258535 non-null  object 
 8   Medicare benefits per 100 people ($)          234301 non-null  float64
 9   No. of patients                               23

In [1186]:
# rename columns for easy references
df_mbs_2014_23.rename(
    columns={
        "Medicare benefits per 100 people ($)": "MBS_per_100",
        "No. of patients": "No_of_patients",
        "No. of services": "No_of_services",
        "Percentage of people who had the service (%)": "%_People_had_service",
        "Services per 100 people": "Services_100_people",
        "Total Medicare benefits paid ($)": "Total_mbs_paid_$",
        "Total provider fees ($)": "Total_provider_fees_$",
        "Estimated resident population": "ERP",
    },
    inplace=True,
)

In [1187]:
# create key for reference and comparison
df_mbs_2014_23["key"] = (
    df_mbs_2014_23["Year"].astype("str") + "-" + df_mbs_2014_23["GeographicCode"]
)
df_mbs_2014_23.head(3)

,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,DemographicGroup,MBS_per_100,No_of_patients,No_of_services,%_People_had_service,Services_100_people,Total_mbs_paid_$,Total_provider_fees_$,ERP,key
0,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 1,Allied Health attendances (total),0-24,2576.0,5624,10879,17.27,33.41,838549.0,1026474.0,32558,2014-80101
1,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 1,Allied Health attendances (total),25-44,4004.0,7714,15870,24.75,50.93,1247656.0,1600846.0,31163,2014-80101
2,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 1,Allied Health attendances (total),45-64,4672.0,8998,15754,41.32,72.35,1017264.0,1197133.0,21774,2014-80101


### Derive MBS Specific New Variables

#### Out of Pocket

In [1188]:
df_mbs_2014_23["Out_of_Pocket"] = (
    df_mbs_2014_23["Total_provider_fees_$"] - df_mbs_2014_23["Total_mbs_paid_$"]
)
df_mbs_2014_23[
    ["Out_of_Pocket", "Total_provider_fees_$", "Total_mbs_paid_$"]
].value_counts(dropna=False)

Out_of_Pocket  Total_provider_fees_$  Total_mbs_paid_$
NaN            NaN                    NaN                 24234
0.0            0.0                    0.0                   341
               2067.0                 2067.0                  5
               5839.0                 5839.0                  4
               17371.0                17371.0                 4
                                                          ...  
7264.0         78258.0                70994.0                 1
               99492.0                92228.0                 1
7265.0         24435.0                17170.0                 1
7266.0         18741.0                11475.0                 1
41936.0        2329262.0              2287326.0               1
Length: 232767, dtype: int64

In [1189]:
df_mbs_2014_23[df_mbs_2014_23["Out_of_Pocket"].isnull()]

,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,DemographicGroup,MBS_per_100,No_of_patients,No_of_services,%_People_had_service,Services_100_people,Total_mbs_paid_$,Total_provider_fees_$,ERP,key,Out_of_Pocket
48,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 3,Diabetes Education,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,97112,2014-80101,NaN
64,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 3,GP Prolonged - Imminent danger of death,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,97112,2014-80101,NaN
67,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 3,GP Telehealth (patient-end support),All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,97112,2014-80101,NaN
70,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 3,Midwifery,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,97112,2014-80101,NaN
74,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 3,Other Allied Health,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,97112,2014-80101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258503,2022,WA,51104,Mid West,Remote (incl. very remote),Level 3,Exercise Physiology,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,57284,2022-51104,NaN
258504,2022,WA,51104,Mid West,Remote (incl. very remote),Level 3,GP Acupuncture,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,57284,2022-51104,NaN
258508,2022,WA,51104,Mid West,Remote (incl. very remote),Level 3,GP Focussed Psychological Strategies and Famil...,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,57284,2022-51104,NaN
258524,2022,WA,51104,Mid West,Remote (incl. very remote),Level 3,Osteopathy,All persons,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,57284,2022-51104,NaN


In [1190]:
# checking for any out of pocket cost that is empty and provider or mbs fees values are not empty
df_mbs_2014_23[
    (df_mbs_2014_23["Out_of_Pocket"].isnull())
    & (
        (~df_mbs_2014_23["Total_mbs_paid_$"].isnull())
        | (~df_mbs_2014_23["Out_of_Pocket"].isnull())
    )
]

,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,DemographicGroup,MBS_per_100,No_of_patients,No_of_services,%_People_had_service,Services_100_people,Total_mbs_paid_$,Total_provider_fees_$,ERP,key,Out_of_Pocket


#### Percentage of Out of Pocket of Provider Fees

Calculate percentage of out of pocket costs paid by patients

In [1191]:
# derive out of pocket cost % value
df_mbs_2014_23["Out_of_pocket_cost_%"] = (
    df_mbs_2014_23["Out_of_Pocket"] / df_mbs_2014_23["Total_provider_fees_$"]
) * 100

# display specific columns for checking
df_mbs_2014_23[
    [
        "Year",
        "GeographicCode",
        "Service",
        "Out_of_Pocket",
        "Total_provider_fees_$",
        "Out_of_pocket_cost_%",
    ]
]

,Year,GeographicCode,Service,Out_of_Pocket,Total_provider_fees_$,Out_of_pocket_cost_%
0,2014,80101,Allied Health attendances (total),187925.0,1026474.0,18.307819
1,2014,80101,Allied Health attendances (total),353190.0,1600846.0,22.062709
2,2014,80101,Allied Health attendances (total),179869.0,1197133.0,15.024981
3,2014,80101,Allied Health attendances (total),85891.0,761837.0,11.274196
4,2014,80101,Allied Health attendances (total),806875.0,4586290.0,17.593196
...,...,...,...,...,...,...
258530,2022,51104,Physiotherapy,17168.0,154790.0,11.091156
258531,2022,51104,Podiatry,41677.0,310628.0,13.417013
258532,2022,51104,Practice Nurse/Aboriginal Health Worker,8.0,133983.0,0.005971
258533,2022,51104,Psychiatry,188594.0,567161.0,33.252286


In [1192]:
# checking for nulls
df_mbs_2014_23[
    ["Out_of_Pocket", "Total_provider_fees_$", "Out_of_pocket_cost_%"]
].value_counts(dropna=False)

Out_of_Pocket  Total_provider_fees_$  Out_of_pocket_cost_%
NaN            NaN                    NaN                     24234
0.0            0.0                    NaN                       341
               2067.0                 0.000000                    5
               5839.0                 0.000000                    4
               17371.0                0.000000                    4
                                                              ...  
7264.0         78258.0                9.282118                    1
               99492.0                7.301090                    1
7265.0         24435.0                29.731942                   1
7266.0         18741.0                38.770610                   1
41936.0        2329262.0              1.800399                    1
Length: 232767, dtype: int64

#### Number of Service Per person

In [1193]:
# derive number of service per person
df_mbs_2014_23["No_of_service_per_person"] = (
    df_mbs_2014_23["No_of_services"] / df_mbs_2014_23["No_of_patients"]
)

df_mbs_2014_23[
    [
        "Year",
        "GeographicCode",
        "Service",
        "No_of_services",
        "No_of_patients",
        "No_of_service_per_person",
    ]
]

,Year,GeographicCode,Service,No_of_services,No_of_patients,No_of_service_per_person
0,2014,80101,Allied Health attendances (total),10879,5624,1.934388
1,2014,80101,Allied Health attendances (total),15870,7714,2.057298
2,2014,80101,Allied Health attendances (total),15754,8998,1.750834
3,2014,80101,Allied Health attendances (total),12316,6397,1.925277
4,2014,80101,Allied Health attendances (total),54818,28733,1.907841
...,...,...,...,...,...,...
258530,2022,51104,Physiotherapy,2485,922,2.695228
258531,2022,51104,Podiatry,4856,1803,2.693289
258532,2022,51104,Practice Nurse/Aboriginal Health Worker,8167,5139,1.58922
258533,2022,51104,Psychiatry,1757,788,2.229695


In [1194]:
# checking for valid null
df_mbs_2014_23[
    [
        "No_of_services",
        "No_of_patients",
        "No_of_service_per_person",
    ]
].value_counts(dropna=False)

No_of_services  No_of_patients  No_of_service_per_person
<NA>            <NA>            <NA>                        24234
0               0               NaN                           341
21              21              1.0                           106
28              28              1.0                            99
26              26              1.0                            96
                                                            ...  
4607            2430            1.895885                        1
                2711            1.699373                        1
                3061            1.505064                        1
                3102            1.485171                        1
10179           6521            1.560957                        1
Length: 211503, dtype: int64

### MBS: Subset National & State Level Data

Subset MBS data by separating State from National level data. This done by creating 3 new dataset containing:
1. National-SA3Group
2. National
3. State

Rows associalted with State Territory National-SA3Group and National will be removed from State MBS dataset

In [1195]:
df_mbs_2014_23["StateTerritory"].value_counts()

NSW                  68503
Qld                  61334
Vic                  49367
WA                   25414
SA                   20941
Tas                  11210
ACT                   7480
NT                    6721
National-SA3Group     4488
Other Territories     2827
National               250
Name: StateTerritory, dtype: int64

#### Subset National Level Data

In [1196]:
# create subset containing dataset National level. Not this only has data from 2020-2022
df_national_au = df_mbs_2014_23[df_mbs_2014_23["StateTerritory"].isin(["National"])]
print(df_national_au.shape)
print(df_national_au["Year"].value_counts(dropna=False))

(250, 20)
2022    84
2020    83
2021    83
Name: Year, dtype: int64


#### Subset SA3 Group Level Data

In [1197]:
# subset data by National-SA3 Group. These were rows with State value that were updated in script 1 before pivotinig
df_sa3_group_au = df_mbs_2014_23[
    df_mbs_2014_23["StateTerritory"].isin(["National-SA3Group"])
]
print(df_sa3_group_au.shape)
print(df_sa3_group_au["Year"].value_counts(dropna=False))

(4488, 20)
2022    504
2014    498
2015    498
2016    498
2017    498
2018    498
2019    498
2020    498
2021    498
Name: Year, dtype: int64


There are 6 more rows in 2022 than other years. Below investigation found "Other GP Services" was added in 2022 service level 3 list

In [1198]:
# checking unique count per geographic code. Appears there is an extra service row for each area
df_sa3_group_au[["Year", "GeographicCode"]].value_counts(dropna=False)

Year  GeographicCode
2022  004-06            84
      004-05            84
      004-04            84
      004-03            84
      004-02            84
      004-01            84
2020  004-03            83
2018  004-06            83
2019  004-01            83
      004-02            83
      004-03            83
      004-04            83
      004-05            83
      004-06            83
2020  004-01            83
      004-02            83
      004-06            83
      004-04            83
      004-05            83
2014  004-02            83
2021  004-01            83
      004-02            83
      004-03            83
      004-04            83
      004-05            83
      004-06            83
2018  004-05            83
2014  004-01            83
2018  004-03            83
      004-02            83
2014  004-03            83
      004-04            83
      004-05            83
      004-06            83
2015  004-01            83
      004-02            83
      0

In [1199]:
# extracting unique list of services for 2021 and 2022 to determine service new service added in 2022
df_service_level_3_2021 = (
    df_sa3_group_au[
        (df_sa3_group_au["Year"] == 2021)
        & (df_sa3_group_au["ServiceLevel"] == "Level 3")
    ]["Service"]
).unique()
df_service_level_3_2022 = (
    df_sa3_group_au[
        (df_sa3_group_au["Year"] == 2022)
        & (df_sa3_group_au["ServiceLevel"] == "Level 3")
    ]["Service"]
).unique()

# finding the difference in service list
np.setdiff1d(df_service_level_3_2022, df_service_level_3_2021)

array(['Other GP Services'], dtype=object)

### Subset State Level SA3 Data

In [1200]:
# remove national and national-sagroup data from masterset to create a state only dataset
df_mbs_state_sa3_complete = df_mbs_2014_23[
    ~df_mbs_2014_23["StateTerritory"].isin(["National", "National-SA3Group"])
]
df_mbs_state_sa3_complete.shape

(253797, 20)

In [1201]:
df_mbs_state_sa3_complete["StateTerritory"].value_counts(dropna=False)

NSW                  68503
Qld                  61334
Vic                  49367
WA                   25414
SA                   20941
Tas                  11210
ACT                   7480
NT                    6721
Other Territories     2827
Name: StateTerritory, dtype: int64

In [1202]:
df_mbs_state_sa3_complete["GeographicCode"].value_counts(dropna=False)

80101    748
40203    748
40703    748
40702    748
40701    748
        ... 
90104    720
10803    691
90101    691
90102    668
12402    498
Name: GeographicCode, Length: 340, dtype: int64

In [1203]:
print(df_mbs_state_sa3_complete["Year"].min())
print(df_mbs_state_sa3_complete["Year"].max())

2014
2022


### Pivot State MBS by Demograpghic Data

In [1204]:
# find unique demographic values
df_mbs_state_sa3_complete["DemographicGroup"].unique()

array(['0-24', '25-44', '45-64', '65+', 'All persons', 'Females', 'Males'],
      dtype=object)

In [1205]:
df_mbs_state_sa3_complete.columns

Index(['Year', 'StateTerritory', 'GeographicCode', 'GeographicAreaName',
       'GeographicGroup', 'ServiceLevel', 'Service', 'DemographicGroup',
       'MBS_per_100', 'No_of_patients', 'No_of_services',
       '%_People_had_service', 'Services_100_people', 'Total_mbs_paid_$',
       'Total_provider_fees_$', 'ERP', 'key', 'Out_of_Pocket',
       'Out_of_pocket_cost_%', 'No_of_service_per_person'],
      dtype='object')

In [1206]:
# Use pivot_table to reshape the DataFrame
df_mbs_state_sa3_pivot = df_mbs_state_sa3_complete.pivot_table(
    index=[
        "key",
        "Year",
        "StateTerritory",
        "GeographicCode",
        "GeographicAreaName",
        "GeographicGroup",
        "ServiceLevel",
        "Service",
    ],
    columns="DemographicGroup",
    values=[
        "MBS_per_100",
        "No_of_patients",
        "No_of_services",
        "%_People_had_service",
        "Services_100_people",
        "Total_mbs_paid_$",
        "Total_provider_fees_$",
        "ERP",
        "Out_of_Pocket",
        "Out_of_pocket_cost_%",
        "No_of_service_per_person",
    ],
    aggfunc="first",
)  # 'first' is used to pick the first value in case of duplicates
df_mbs_state_sa3_pivot.reset_index(inplace=True)

In [1207]:
# Flatten the MultiIndex in columns and format new column names as 'demographic_value'
df_mbs_state_sa3_pivot.columns = [
    "_".join(col).strip() if col[1] else col[0]
    for col in df_mbs_state_sa3_pivot.columns.values
]

In [1208]:
df_mbs_state_sa3_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162087 entries, 0 to 162086
Data columns (total 85 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   key                                   162087 non-null  object 
 1   Year                                  162087 non-null  int64  
 2   StateTerritory                        162087 non-null  object 
 3   GeographicCode                        162087 non-null  object 
 4   GeographicAreaName                    162087 non-null  object 
 5   GeographicGroup                       162087 non-null  object 
 6   ServiceLevel                          162087 non-null  object 
 7   Service                               162087 non-null  object 
 8   %_People_had_service_0-24             14724 non-null   float64
 9   %_People_had_service_25-44            14802 non-null   float64
 10  %_People_had_service_45-64            14767 non-null   float64
 11  

In [1209]:
df_mbs_state_sa3_pivot.head(3)

,key,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,%_People_had_service_0-24,%_People_had_service_25-44,...,Total_mbs_paid_$_All persons,Total_mbs_paid_$_Females,Total_mbs_paid_$_Males,Total_provider_fees_$_0-24,Total_provider_fees_$_25-44,Total_provider_fees_$_45-64,Total_provider_fees_$_65+,Total_provider_fees_$_All persons,Total_provider_fees_$_Females,Total_provider_fees_$_Males
0,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Allied Health attendances (total),18.80,20.65,...,1938341.0,1201578.0,736763.0,580491.0,633972.0,717555.0,360781.0,2292799.0,1428964.0,863835.0
1,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Diagnostic Imaging (total),17.59,27.72,...,4702507.0,2761452.0,1941056.0,657658.0,1586180.0,2387889.0,1558873.0,6190599.0,3768680.0,2421920.0
2,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,GP attendances (total),76.02,74.84,...,10780077.0,6339946.0,4440131.0,3207438.0,3764233.0,4500909.0,3067848.0,14540427.0,8587817.0,5952610.0


In [1210]:
# checking the pivot values
df_mbs_state_sa3_pivot[
    (df_mbs_state_sa3_pivot["Year"] == 2014)
    & (df_mbs_state_sa3_pivot["GeographicCode"] == "80101")
    & (df_mbs_state_sa3_pivot["ServiceLevel"] == "Level 1")
    & (
        df_mbs_state_sa3_pivot["Service"].isin(
            ["Allied Health attendances (total)", "Diagnostic Imaging (total)"]
        )
    )
]

,key,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,%_People_had_service_0-24,%_People_had_service_25-44,...,Total_mbs_paid_$_All persons,Total_mbs_paid_$_Females,Total_mbs_paid_$_Males,Total_provider_fees_$_0-24,Total_provider_fees_$_25-44,Total_provider_fees_$_45-64,Total_provider_fees_$_65+,Total_provider_fees_$_All persons,Total_provider_fees_$_Females,Total_provider_fees_$_Males
17278,2014-80101,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 1,Allied Health attendances (total),17.27,24.75,...,3779415.0,2394671.0,1384744.0,1026474.0,1600846.0,1197133.0,761837.0,4586290.0,2929743.0,1656547.0
17279,2014-80101,2014,ACT,80101,Belconnen,Major cities - medium SES,Level 1,Diagnostic Imaging (total),17.18,30.48,...,8274560.0,5024192.0,3250368.0,1051718.0,3173663.0,3601039.0,3040462.0,10866882.0,6807431.0,4059451.0


Note, there are is no data by demograpgic for service level 2 & 3. Due to this, significant number of blank values exists in measure_value_demographic columns except for all person

In [1211]:
# exporting dataset for analysis of the pivot
df_mbs_state_sa3_pivot.to_csv(
    os.path.join(path, "clean_datasets/mbs_data/mbs_state_pivot_datatset_complete.csv")
)

## Import & Ready Census Dataset

In [1212]:
# import the transformed mbs file and assign to a dataframe

df_census_2011_22 = pd.read_pickle(
    os.path.join(path, "clean_datasets/census_data/2011_22_census_complete.pkl")
)
df_census_2011_22.info(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4296 entries, 0 to 4295
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SA3                      4296 non-null   object 
 1   Year                     4296 non-null   int64  
 2   age_0-14                 4296 non-null   float64
 3   age_15-24                4296 non-null   float64
 4   age_25-44                4296 non-null   float64
 5   age_45-64                4296 non-null   float64
 6   age_65-79                4296 non-null   float64
 7   age_80+                  4296 non-null   float64
 8   negative_income          4296 non-null   float64
 9   no_income                4296 non-null   float64
 10  average_income_$5200     4296 non-null   float64
 11  average_income_$13000    4296 non-null   float64
 12  average_income_$18200    4296 non-null   float64
 13  average_income_$26000    4296 non-null   float64
 14  average_income_$36400   

### Extract data from 2014

In [1213]:
# extracting data from 2014 onwards to match the MBS range
df_census_2014_22 = df_census_2011_22[df_census_2011_22["Year"] >= 2014]
df_census_2014_22["Year"].value_counts(dropna=False)

2014    358
2015    358
2016    358
2017    358
2018    358
2019    358
2020    358
2021    358
2022    358
Name: Year, dtype: int64

### Align Age Groups with MBS

1. 0-24
2. 25-44
3. 45-64
4. 65+

In [1214]:
# make a copy of the dataset to avoid changing the original imported version
df_census_2014_22_new = df_census_2014_22.copy()

In [1215]:
# combine population of age bracks 0-14 and 15-24 to create a 0-24 bracket
df_census_2014_22_new["age_0-24"] = (
    df_census_2014_22_new["age_0-14"] + df_census_2014_22_new["age_15-24"]
)
# combine population of age bracks 65-79 and 8-+ to create a 65+ bracket
df_census_2014_22_new["age_65+"] = (
    df_census_2014_22_new["age_65-79"] + df_census_2014_22_new["age_80+"]
)

In [1216]:
df_census_2014_22_new.head(3)

,SA3,Year,age_0-14,age_15-24,age_25-44,age_45-64,age_65-79,age_80+,negative_income,no_income,...,average_income_$91000,average_income_$130000,not_stated,not_applicable,male_pop,female_pop,total_population,average_income_$169000+,age_0-24,age_65+
3,10102,2014,11151.0,6831.0,15725.0,15715.2,5045.8,1332.6,166.2,2839.0,...,5117.2,4088.6,3787.6,11151.0,28038.8,27765.2,55804.2,NaN,17982.0,6378.4
4,10102,2015,11193.0,6854.0,15782.0,16033.6,5337.4,1363.8,163.6,2951.0,...,5324.6,4089.8,4047.8,11193.0,28434.4,28133.6,56568.6,NaN,18047.0,6701.2
5,10102,2016,11235.0,6877.0,15839.0,16352.0,5629.0,1395.0,161.0,3063.0,...,5532.0,4091.0,4308.0,11235.0,28830.0,28502.0,57333.0,1812.0,18112.0,7024.0


In [1217]:
# Drop columns age_0-14, age_15-24, age_65-79, age_80+ as aggregate columns are formed

df_census_2014_22_new.drop(
    ["age_0-14", "age_15-24", "age_65-79", "age_80+"], axis=1, inplace=True
)
df_census_2014_22_new.columns

Index(['SA3', 'Year', 'age_25-44', 'age_45-64', 'negative_income', 'no_income',
       'average_income_$5200', 'average_income_$13000',
       'average_income_$18200', 'average_income_$26000',
       'average_income_$36400', 'average_income_$46800',
       'average_income_$58500', 'average_income_$71500',
       'average_income_$91000', 'average_income_$130000', 'not_stated',
       'not_applicable', 'male_pop', 'female_pop', 'total_population',
       'average_income_$169000+', 'age_0-24', 'age_65+'],
      dtype='object')

### Compare SA3 values

Found SA3 values exists in census but not in MBS. This is expected and is not an issue for the join

Make census + year key to find the missing year-census 

In [1218]:
# creating key for reference and investigations
df_census_2014_22_new["key"] = (
    df_census_2014_22_new["Year"].astype("str") + "-" + df_census_2014_22_new["SA3"]
)

In [1219]:
df_mbs_state_sa3_pivot.head(3)

,key,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,%_People_had_service_0-24,%_People_had_service_25-44,...,Total_mbs_paid_$_All persons,Total_mbs_paid_$_Females,Total_mbs_paid_$_Males,Total_provider_fees_$_0-24,Total_provider_fees_$_25-44,Total_provider_fees_$_45-64,Total_provider_fees_$_65+,Total_provider_fees_$_All persons,Total_provider_fees_$_Females,Total_provider_fees_$_Males
0,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Allied Health attendances (total),18.80,20.65,...,1938341.0,1201578.0,736763.0,580491.0,633972.0,717555.0,360781.0,2292799.0,1428964.0,863835.0
1,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Diagnostic Imaging (total),17.59,27.72,...,4702507.0,2761452.0,1941056.0,657658.0,1586180.0,2387889.0,1558873.0,6190599.0,3768680.0,2421920.0
2,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,GP attendances (total),76.02,74.84,...,10780077.0,6339946.0,4440131.0,3207438.0,3764233.0,4500909.0,3067848.0,14540427.0,8587817.0,5952610.0


In [1220]:
census_sa3_list = pd.Series(df_census_2014_22_new["key"].unique())
mbs_sa3_list = pd.Series(df_mbs_state_sa3_pivot["key"].unique())

census_sa3_list.to_clipboard()

In [1221]:
# diff_values = np.setdiff1d(census_sa3_list, mbs_sa3_list)
diff_values = mbs_sa3_list[~mbs_sa3_list.isin(census_sa3_list)]
diff_values

Series([], dtype: object)

Diff_values shows keys in mbs also exists in census

In [1222]:
# checking population data of SA3 codes added in 2016 and backfilled values
df_census_2014_22_new[
    (
        df_census_2014_22_new["Year"].isin([2014, 2015])
        & (df_census_2014_22_new["SA3"] == "90104")
    )
]

,SA3,Year,age_25-44,age_45-64,negative_income,no_income,average_income_$5200,average_income_$13000,average_income_$18200,average_income_$26000,...,average_income_$130000,not_stated,not_applicable,male_pop,female_pop,total_population,average_income_$169000+,age_0-24,age_65+,key
4263,90104,2014,341.0,577.0,9.0,67.0,54.0,98.0,146.0,344.0,...,33.0,138.0,296.0,820.0,925.0,1748.0,NaN,406.0,413.0,2014-90104
4264,90104,2015,341.0,577.0,9.0,67.0,54.0,98.0,146.0,344.0,...,33.0,138.0,296.0,820.0,925.0,1748.0,NaN,406.0,413.0,2015-90104


## Combine MBS and Census

Combining MBS state complete dataset with census. Expected 162,087 rows, 110 columns. The join will occur on SA3 (GeographicCode) and Year

In [1223]:
df_mbs_state_sa3_pivot.shape

(162087, 85)

In [1224]:
df_census_2014_22_new.shape

(3222, 25)

In [1225]:
df_census_mbs_combined = df_mbs_state_sa3_pivot.merge(
    df_census_2014_22_new,
    how="left",
    left_on=["Year", "GeographicCode"],
    right_on=["Year", "SA3"],
    indicator=True,
)

In [1226]:
df_census_mbs_combined.shape

(162087, 110)

In [1227]:
df_census_mbs_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162087 entries, 0 to 162086
Columns: 110 entries, key_x to _merge
dtypes: Float64(7), Int32(7), Int64(14), category(1), float64(71), int64(1), object(9)
memory usage: 136.2+ MB


In [1228]:
df_census_mbs_combined.isnull().sum()

key_x                                        0
Year                                         0
StateTerritory                               0
GeographicCode                               0
GeographicAreaName                           0
GeographicGroup                              0
ServiceLevel                                 0
Service                                      0
%_People_had_service_0-24               147363
%_People_had_service_25-44              147285
%_People_had_service_45-64              147320
%_People_had_service_65+                147345
%_People_had_service_All persons         21276
%_People_had_service_Females            147226
%_People_had_service_Males              147225
ERP_0-24                                146812
ERP_25-44                               146802
ERP_45-64                               146817
ERP_65+                                 146807
ERP_All persons                              0
ERP_Females                             146802
ERP_Males    

Investigated the lot of nulls. This is due to Service Level 2 and 3 not having gender and age bracket information. Because of this, for the analysis, splitting data into 3 datasets.

1. 2014_22_mbs_census_combined : full combined dataset
2. 2014_22_mbs_census_service_level_1
3. 2014_22_mbs_census_service_level_2_3

In [1229]:
# export complete census and mbs data combined.
df_census_mbs_combined.to_pickle(
    os.path.join(path, "clean_datasets/2014_22_mbs_census_combined.pkl")
)

#### Subset Service Level 1

In [1230]:
# subsetting service level 1 dataset
df_mbs_census_service_level_1 = df_census_mbs_combined[
    df_census_mbs_combined["ServiceLevel"] == "Level 1"
]
df_mbs_census_service_level_1.shape

(15285, 110)

In [1231]:
df_mbs_census_service_level_1.isnull().sum()

key_x                                      0
Year                                       0
StateTerritory                             0
GeographicCode                             0
GeographicAreaName                         0
GeographicGroup                            0
ServiceLevel                               0
Service                                    0
%_People_had_service_0-24                561
%_People_had_service_25-44               483
%_People_had_service_45-64               518
%_People_had_service_65+                 543
%_People_had_service_All persons         420
%_People_had_service_Females             424
%_People_had_service_Males               423
ERP_0-24                                  10
ERP_25-44                                  0
ERP_45-64                                 15
ERP_65+                                    5
ERP_All persons                            0
ERP_Females                                0
ERP_Males                                  0
MBS_per_10

In [1232]:
df_mbs_census_service_level_1[df_mbs_census_service_level_1["Year"] == 2014]

,key_x,Year,StateTerritory,GeographicCode,GeographicAreaName,GeographicGroup,ServiceLevel,Service,%_People_had_service_0-24,%_People_had_service_25-44,...,not_stated,not_applicable,male_pop,female_pop,total_population,average_income_$169000+,age_0-24,age_65+,key_y,_merge
0,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Allied Health attendances (total),18.80,20.65,...,3787.6,11151.0,28038.8,27765.2,55804.2,NaN,17982.0,6378.4,2014-10102,both
1,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Diagnostic Imaging (total),17.59,27.72,...,3787.6,11151.0,28038.8,27765.2,55804.2,NaN,17982.0,6378.4,2014-10102,both
2,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,GP attendances (total),76.02,74.84,...,3787.6,11151.0,28038.8,27765.2,55804.2,NaN,17982.0,6378.4,2014-10102,both
3,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Nursing and Aboriginal Health Workers (total),1.14,0.59,...,3787.6,11151.0,28038.8,27765.2,55804.2,NaN,17982.0,6378.4,2014-10102,both
4,2014-10102,2014,NSW,10102,Queanbeyan,Major cities - higher SES,Level 1,Specialist attendances (total),14.60,17.92,...,3787.6,11151.0,28038.8,27765.2,55804.2,NaN,17982.0,6378.4,2014-10102,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17967,2014-90104,2014,Other Territories,90104,Norfolk Island,Remote (incl. very remote),Level 1,Allied Health attendances (total),NaN,NaN,...,138.0,296.0,820.0,925.0,1748.0,NaN,406.0,413.0,2014-90104,both
17968,2014-90104,2014,Other Territories,90104,Norfolk Island,Remote (incl. very remote),Level 1,Diagnostic Imaging (total),NaN,NaN,...,138.0,296.0,820.0,925.0,1748.0,NaN,406.0,413.0,2014-90104,both
17969,2014-90104,2014,Other Territories,90104,Norfolk Island,Remote (incl. very remote),Level 1,GP attendances (total),NaN,NaN,...,138.0,296.0,820.0,925.0,1748.0,NaN,406.0,413.0,2014-90104,both
17970,2014-90104,2014,Other Territories,90104,Norfolk Island,Remote (incl. very remote),Level 1,Nursing and Aboriginal Health Workers (total),NaN,NaN,...,138.0,296.0,820.0,925.0,1748.0,NaN,406.0,413.0,2014-90104,both


In [1233]:
# checking 2016 new SA3 codes having backfilled population values
old_sa3_codes = df_mbs_census_service_level_1[
    (
        df_mbs_census_service_level_1["GeographicCode"].isin(
            ["80110", "80111", "10106", "90104", "30805", "31608", "21704", "51003"]
        )
    )
    & (df_mbs_census_service_level_1["total_population"].isnull())
]
old_sa3_codes["Year"].value_counts(dropna=False)

Series([], Name: Year, dtype: int64)

In [1234]:
df_mbs_census_service_level_1_new = df_mbs_census_service_level_1.drop(
    columns=["key_y", "_merge", "SA3"]
)

In [1235]:
# Exporting df_mbs_census_service_level_1 into pickle file
df_mbs_census_service_level_1_new.to_pickle(
    os.path.join(path, "clean_datasets/2014_22_mbs_census_service_level_1.pkl")
)

#### Subset Service Level 2 & 3

In [1236]:
# subsetting service level 2 and 3 dataset
df_mbs_census_service_level_2_3 = df_census_mbs_combined[
    df_census_mbs_combined["ServiceLevel"].isin(["Level 2", "Level 3"])
]
df_mbs_census_service_level_2_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146802 entries, 5 to 162086
Columns: 110 entries, key_x to _merge
dtypes: Float64(7), Int32(7), Int64(14), category(1), float64(71), int64(1), object(9)
memory usage: 123.3+ MB


In [1237]:
df_mbs_census_service_level_2_3.columns

Index(['key_x', 'Year', 'StateTerritory', 'GeographicCode',
       'GeographicAreaName', 'GeographicGroup', 'ServiceLevel', 'Service',
       '%_People_had_service_0-24', '%_People_had_service_25-44',
       ...
       'not_stated', 'not_applicable', 'male_pop', 'female_pop',
       'total_population', 'average_income_$169000+', 'age_0-24', 'age_65+',
       'key_y', '_merge'],
      dtype='object', length=110)

In [1238]:
df_mbs_census_service_level_2_3_new = df_mbs_census_service_level_2_3.copy()
df_mbs_census_service_level_2_3_new.drop(
    columns=[
        "Total_provider_fees_$_Males",
        "Total_provider_fees_$_Females",
        "Total_provider_fees_$_65+",
        "Total_provider_fees_$_45-64",
        "Total_provider_fees_$_25-44",
        "Total_provider_fees_$_0-24",
        "Services_100_people_Males",
        "Services_100_people_Females",
        "Services_100_people_65+",
        "Services_100_people_45-64",
        "Services_100_people_25-44",
        "Services_100_people_0-24",
        "No_of_services_Males",
        "No_of_services_Females",
        "No_of_services_0-24",
        "No_of_services_25-44",
        "No_of_services_45-64",
        "No_of_services_65+",
        "%_People_had_service_0-24",
        "%_People_had_service_25-44",
        "%_People_had_service_45-64",
        "%_People_had_service_65+",
        "%_People_had_service_Females",
        "%_People_had_service_Males",
        "ERP_0-24",
        "ERP_25-44",
        "ERP_45-64",
        "ERP_65+",
        "ERP_Females",
        "ERP_Males",
        "MBS_per_100_0-24",
        "MBS_per_100_25-44",
        "MBS_per_100_45-64",
        "MBS_per_100_65+",
        "MBS_per_100_Females",
        "MBS_per_100_Males",
        "No_of_patients_0-24",
        "No_of_patients_25-44",
        "No_of_patients_45-64",
        "No_of_patients_65+",
        "No_of_patients_Females",
        "No_of_patients_Males",
        "Total_mbs_paid_$_0-24",
        "Total_mbs_paid_$_25-44",
        "Total_mbs_paid_$_45-64",
        "Total_mbs_paid_$_65+",
        "Total_mbs_paid_$_Females",
        "Total_mbs_paid_$_Males",
        "_merge",
        "SA3",
        "key_y",
        "Out_of_Pocket_0-24",
        "Out_of_Pocket_25-44",
        "Out_of_Pocket_45-64",
        "Out_of_Pocket_65+",
        "Out_of_Pocket_Females",
        "Out_of_Pocket_Males",
        "Out_of_pocket_cost_%_0-24",
        "Out_of_pocket_cost_%_25-44",
        "Out_of_pocket_cost_%_45-64",
        "Out_of_pocket_cost_%_65+",
        "Out_of_pocket_cost_%_Females",
        "Out_of_pocket_cost_%_Males",
        "No_of_service_per_person_0-24",
        "No_of_service_per_person_25-44",
        "No_of_service_per_person_45-64",
        "No_of_service_per_person_65+",
        "No_of_service_per_person_Females",
        "No_of_service_per_person_Males",
    ],
    inplace=True,
)
df_mbs_census_service_level_2_3_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146802 entries, 5 to 162086
Data columns (total 41 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   key_x                                 146802 non-null  object 
 1   Year                                  146802 non-null  int64  
 2   StateTerritory                        146802 non-null  object 
 3   GeographicCode                        146802 non-null  object 
 4   GeographicAreaName                    146802 non-null  object 
 5   GeographicGroup                       146802 non-null  object 
 6   ServiceLevel                          146802 non-null  object 
 7   Service                               146802 non-null  object 
 8   %_People_had_service_All persons      125946 non-null  float64
 9   ERP_All persons                       146802 non-null  Int64  
 10  MBS_per_100_All persons               125946 non-null  float64
 11  

In [1239]:
# Exporting df_mbs_census_service_level_1 into pickle file
df_mbs_census_service_level_2_3_new.to_pickle(
    os.path.join(path, "clean_datasets/2014_22_mbs_census_service_level_2_3.pkl")
)